In [39]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import string
import os
import shutil

In [13]:
company_names = pd.read_csv('../Company Names.csv')
company_names

<IPython.core.display.Javascript object>

,Company
0,2U INC
1,30DC INC
2,3D PIONEER SYSTEMS INC
3,8X8 INC
4,A10 NETWORKS INC
...,...
287,ZOOM VIDEO COMUNICATIONS INC
288,ZOOMAWAY TRAVEL INC
289,ZOOMINFO TECHNOLOGIES
290,ZSCALER INC


In [27]:
columns_cik_numbers = ['Company','CIK','NaN']
cik_numbers = pd.read_csv('../CIK.txt',sep = ':',header = None,names = columns_cik_numbers)
cik_numbers = cik_numbers.drop(columns='NaN')
cik_numbers.head()

<IPython.core.display.Javascript object>

C:\Users\91988\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Company,CIK
0,!J INC,0001438823
1,"#1 A LIFESAFER HOLDINGS, INC.",0001509607
2,#1 ARIZONA DISCOUNT PROPERTIES LLC,0001457512
3,#1 PAINTBALL CORP,0001433777
4,$ LLC,0001427189


In [30]:
punctuations = string.punctuation
cik_numbers['Company'] = cik_numbers['Company'].apply(lambda x: str(x).translate(str.maketrans('','',punctuations)))
cik_numbers['Company'] = cik_numbers['Company'].apply(lambda x: str(x).upper())

company_names['Company'] = company_names['Company'].apply(lambda x: str(x).translate(str.maketrans('','',punctuations)))
company_names['Company'] = company_names['Company'].apply(lambda x: str(x).upper())

required_data = pd.merge(company_names,cik_numbers,on = 'Company')
required_data['CIK'] = required_data['CIK'].apply(lambda x: str(int(x)))
required_data

<IPython.core.display.Javascript object>

,Company,CIK
0,2U INC,1459417
1,30DC INC,1118974
2,3D PIONEER SYSTEMS INC,1556753
3,8X8 INC,1023731
4,A10 NETWORKS INC,1580808
...,...,...
269,ZOOM TELEPHONICS INC,822708
270,ZOOM TELEPHONICS INC,1467761
271,ZOOMAWAY TRAVEL INC,1541054
272,ZSCALER INC,1713683


In [101]:

for company,cik in required_data['Company'][:2],required_data['CIK'][:2]:
    
    os.makedirs('../Data/{}/10-K'.format(company))
    os.makedirs('../Data/{}/10-Q'.format(company))
    os.makedirs('../Data/{}/8-K'.format(company))
    i = 1
    j = 1
    while True:
        
        driver = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe")
        driver.get("https://www.sec.gov/edgar/browse/?CIK={}&owner=exclude".format(cik))

        time.sleep(1)

        driver.find_element(By.XPATH,"//div[@id='filingsStart']/div[2]/div[3]/h5/a").click()

        view_button = driver.find_element(By.XPATH,"//button[@data-group='annualOrQuarterlyReports']")
        driver.implicitly_wait(1)
        ActionChains(driver).move_to_element(view_button).click(view_button).perform()
        try :
            
            form_type = driver.find_element(By.XPATH,"//tbody/tr[{}]/td[1]".format(i)).text
            href = driver.find_element(By.XPATH,"//tbody/tr[{}]/td[2]/div/a[2]".format(i)).get_attribute('href')
            i += 1
        except:
            break
        
        
        try :
            options = webdriver.ChromeOptions()
            download_dir = "C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{}\\{}".format(company,form_type)
            prefs = {"download.default_directory":download_dir}
            options.add_experimental_option("prefs", prefs)
            driver = webdriver.Chrome('C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe',chrome_options=options)

            driver_10s.get(href)
        except:
            continue
            
        acc_number = driver_10s.find_element(By.XPATH,"//div[@id='secNum']").text.split('.')[-1].strip()

        driver_10s.find_element(By.XPATH,"//a[@id='interactiveDataBtn']").click()

        driver_10s.find_element(By.XPATH,"//table/tbody/tr/td/a[2]").click()
        time.sleep(6)
        driver_10s.close()
        
    while True:
        
        driver = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe")
        driver.get("https://www.sec.gov/edgar/browse/?CIK={}&owner=exclude".format(cik))

        time.sleep(1)

        driver.find_element(By.XPATH,"//div[@id='filingsStart']/div[2]/div[2]/h5/a").click()

        view_button = driver.find_element(By.XPATH,"//button[@data-group='currentReports']")
        driver.implicitly_wait(1)
        ActionChains(driver).move_to_element(view_button).click(view_button).perform()
        try :
            
            form_type = driver.find_element(By.XPATH,"//tbody/tr[{}]/td[1]".format(i)).text
            href = driver.find_element(By.XPATH,"//tbody/tr[{}]/td[2]/div/a[2]".format(i)).get_attribute('href')
            i += 1
        except:
            break
        
        
        try :
            options = webdriver.ChromeOptions()
            download_dir = "C:\\Users\\91988\\Desktop\\Projects\\Inter IIT Tech 10.0\\Data\\{}\\{}".format(company,form_type)
            prefs = {"download.default_directory":download_dir}
            
            driver_8k = webdriver.Chrome('C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe',options=options)
            options.add_experimental_option("prefs",prefs)
            driver_8k.get(href)
        except:
            continue
            
        acc_number = driver_8k.find_element(By.XPATH,"//div[@id='secNum']").text.split('.')[-1].strip()

#             print(acc_number,form_type)
        driver_8k.find_element(By.XPATH,"//a[@id='interactiveDataBtn']").click()

        driver_8k.find_element(By.XPATH,"//table/tbody/tr/td/a[2]").click()
        time.sleep(6)
        driver_8k.close()
        
     

SyntaxError: invalid character in identifier (Temp/ipykernel_19720/1228930095.py, line 34)

In [13]:
driver = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe")
driver.get("https://www.sec.gov/cgi-bin/viewer?action=view&cik=1618732&accession_number=0001618732-18-000173&xbrl_type=v")

time.sleep(3)

driver.find_element(By.XPATH,"//td[@colspan='2']/a[2]").click()

username.send_keys("addisonhardley@gmail.com")
with open("password.txt", "r") as f:
    password = f.read()

pword = driver.find_element(By.ID,"password")

pword.send_keys('addisonhardley123456')

driver.find_element_by_xpath("//button[@type='submit']").click()

C:\Users\91988\AppData\Local\Temp/ipykernel_25080/2366850665.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe")


NameError: name 'username' is not defined